In [0]:
!sudo pip install --upgrade beautifulsoup4
!sudo pip install --upgrade lxml

Requirement already up-to-date: beautifulsoup4 in /usr/local/lib/python3.6/dist-packages (4.9.1)
Requirement already up-to-date: lxml in /usr/local/lib/python3.6/dist-packages (4.5.1)


In [0]:
#%run Pre_ini.ipynb

In [0]:
from bs4 import BeautifulSoup,Comment

In [0]:
def extract_text(soup):
        
    blacklist = set([
        '[document]',
        
        #'header',
        'dialog',
        'html',
        'input',
        
        'script',
        'noscript',
        'applet',
        'embed',  
        'object',
        'param',
        
        
        'head',
        'meta',
        'base',
        'basefont',

        'style',
    ])
    
    def is_comment(element): 
        return isinstance(element, Comment)
        
    for comment in soup.find_all(text=is_comment):
        comment.extract()
    
    
    ans=''
    for text in soup.find_all(text=True):
        if text.parent.name in blacklist:
            continue
        ans+=process_string(text)+' '
    
    return ans

In [0]:
no_head=0
no_body=0

beg=time()


i=0
er=0
with ZipFile('data.zip') as myzip:
    files=myzip.namelist()
    with open('forward_docs.txt','w') as out:
        for file_name in files:
            with myzip.open(file_name) as f:
                url=f.readline().decode('utf-8').strip()
                doc_id=get_id[url]
                raw_html=f.read()
                try:
                    html=raw_html.decode('utf-8')
                except UnicodeDecodeError:
                    html=raw_html.decode('latin-1')
                    er+=1
            
            out.write(str(doc_id)+'\t')
            soup=BeautifulSoup(html)
            if soup.head:
                tmp=soup.head.find('meta',{'name':'description'})
                out.write(process_string(tmp['content']) if tmp is not None and tmp.has_attr('content') else '')
                out.write('\t')

                tmp=soup.head.find('meta',{'name':'author'})
                out.write(process_string(tmp['content']) if tmp is not None and tmp.has_attr('content') else '')
                out.write('\t')

                tmp=soup.head.find('meta',{'name':'keywords'})
                out.write(process_string(tmp['content']) if tmp is not None and tmp.has_attr('content') else '')
                out.write('\t')

                tmp=soup.head.title
                out.write(process_string(tmp.text) if tmp is not None else '')
                out.write('\t')
            else:
                out.write('\t'*4)
                no_head+=1

            if soup.body:
                out.write(extract_text(soup.body))
            else:
                no_body+=1
            out.write('\n')

            if i%100==0:
                print(i)
            i+=1

print(no_head,no_body,er)
print((time()-beg)/i)

In [ ]:
def extract_text(soup):
        
    blacklist = set([
        '[document]',
        
        #'header',
        'dialog',
        'html',
        'input',
        
        'script',
        'noscript',
        'applet',
        'embed',  
        'object',
        'param',
        
        
        'head',
        'meta',
        'base',
        'basefont',

        'style',
    ])
    
    def is_comment(element): 
        return isinstance(element, Comment)
        
    for comment in soup.find_all(text=is_comment):
        comment.extract()
    
          
    classes=[
             set(['a']),
             set(['h1','h2']),
             set(['h3','h4','h5','h6']),
             set(['strong','b','em','i','u','dl','ol','ul'])
    ]
    ans=['' for i in range(len(classes)+1)]
    for text in soup.find_all(text=True):
        if text.parent.name in blacklist:
            continue
        ind=0
        for cls in classes:
            if text.parent.name in cls:
                ans[ind]+=string_lem(text)+' '
                break
            ind+=1
        if ind==len(classes):
            ans[ind]+=string_lem(text)+' '
    return reduce(lambda x,y:x+'\t'+y,ans)
    
    
    return ans

In [ ]:
no_head=0
no_body=0

beg=time()


i=0
er=0
with ZipFile('data.zip') as myzip:
    files=myzip.namelist()
    with open('all_docs.txt','w') as out:
        for file_name in files:
            with myzip.open(file_name) as f:
                url=f.readline().decode('utf-8').strip()
                doc_id=get_id[url]
                raw_html=f.read()
                try:
                    html=raw_html.decode('utf-8')
                except UnicodeDecodeError:
                    html=raw_html.decode('latin-1')
                    er+=1
            
            out.write(str(doc_id)+'\t')
            soup=BeautifulSoup(html)
            if soup.head:
                tmp=soup.head.find('meta',{'name':'description'})
                out.write(process_string(tmp['content']) if tmp is not None and tmp.has_attr('content') else '')
                out.write('\t')

                tmp=soup.head.find('meta',{'name':'author'})
                out.write(process_string(tmp['content']) if tmp is not None and tmp.has_attr('content') else '')
                out.write('\t')

                tmp=soup.head.find('meta',{'name':'keywords'})
                out.write(process_string(tmp['content']) if tmp is not None and tmp.has_attr('content') else '')
                out.write('\t')

                tmp=soup.head.title
                out.write(process_string(tmp.text) if tmp is not None else '')
                out.write('\t')
            else:
                out.write('\t'*4)
                no_head+=1

            if soup.body:
                out.write(extract_text(soup.body))
            else:
                no_body+=1
                out.write('\t'*4)
            out.write('\n')

            if i%100==0:
                print(i)
            i+=1

print(no_head,no_body,er)
print((time()-beg)/i)

In [0]:
IDF=defaultdict(int)
with open('forward_docs.txt','r') as f:
    for i,line in enumerate(f):
        words=' '.join(line.strip('\n').split('\t')[1:]).replace('.',' ').split()
        for word in set(words):
            IDF[word]+=1
        if i%500==0:
            print(i)
with open('IDF.pickle','wb') as out:
    pickle.dump(IDF,out)

In [0]:
IDF_lem=defaultdict(int)
with open('forward_docs.txt','r') as f:
    for i,line in enumerate(f):
        words=' '.join(line.strip('\n').split('\t')[1:]).replace('.',' ').split()
        for word in set(map(lambda x:add_cash(x),
                            words)):
            IDF_lem[word]+=1
        if i%500==0:
            print(i)
with open('IDF_lem.pickle','wb') as out:
    pickle.dump(IDF_lem,out)